<a href="https://colab.research.google.com/github/illhammm/Data-in-chem/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas rdkit pubchempy mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.4 MB/s eta 0:00:00
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=3081ce80ef77c0b5a688371f8887da8f301b53929d3dc7aab2fbc83aacb12cd5
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176720 sha256=1a4e6fd7196002dbebe3758b635f62f6302e782fb6c2c7ca47f626f05b39e737
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built pubchempy mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
    

In [2]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from mordred import Calculator, descriptors
import pubchempy as pcp
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression

In [3]:
# Функция для загрузки дескрипторов из PubChem
def get_pubchem_descriptors(smiles):
    try:
        compound = pcp.get_compounds(smiles, 'smiles')
        cid = compound[0].cid
        properties = pcp.get_properties('property', cid)
        descriptors = {prop['Name']: prop['Value'] for prop in properties[0]}
        return descriptors
    except Exception as e:
        print(f"Error fetching descriptors for SMILES {smiles}: {e}")
        return None

In [4]:
# # Функция для получения дополнительных дескрипторов из PubChem
# def get_additional_pubchem_descriptors(smiles):
#     try:
#         # Ваши операции для получения дополнительных дескрипторов из PubChem
#         pass
#     except Exception as e:
#         print(f"Error fetching additional descriptors for SMILES {smiles}: {e}")
#         return None

In [5]:
# Функция для вычисления дескрипторов с помощью Mordred
def calculate_mordred_descriptors(smiles):
    try:
        molecule = Chem.MolFromSmiles(smiles)
        if molecule is None:
            return None
        calc = Calculator(descriptors)
        descriptors = calc(molecule)
        return descriptors.asdict()
    except Exception as e:
        print(f"Error calculating Mordred descriptors for SMILES {smiles}: {e}")
        return None

In [6]:
df = pd.read_csv("qm9_new.csv")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mol_id  20000 non-null  object 
 1   smiles  20000 non-null  object 
 2   A       20000 non-null  float64
 3   B       20000 non-null  float64
 4   C       20000 non-null  float64
 5   mu      20000 non-null  float64
 6   alpha   20000 non-null  float64
 7   homo    20000 non-null  float64
 8   lumo    20000 non-null  float64
 9   gap     20000 non-null  float64
 10  r2      20000 non-null  float64
 11  zpve    20000 non-null  float64
 12  u0      20000 non-null  float64
 13  u298    20000 non-null  float64
 14  h298    20000 non-null  float64
 15  g298    20000 non-null  float64
 16  cv      20000 non-null  float64
dtypes: float64(15), object(2)
memory usage: 2.6+ MB


In [8]:
# Выборка подмножества данных для тестирования
df = df.head(100)

In [9]:
# Вычисление дескрипторов с помощью Mordred
df['mordred_descriptors'] = df['smiles'].apply(calculate_mordred_descriptors)

Error calculating Mordred descriptors for SMILES OC1C2OC(=N)C12C#C: local variable 'descriptors' referenced before assignment
Error calculating Mordred descriptors for SMILES C1C2OC3C4C=CC13C24: local variable 'descriptors' referenced before assignment
Error calculating Mordred descriptors for SMILES COC(=N)N1C=NN=C1: local variable 'descriptors' referenced before assignment
Error calculating Mordred descriptors for SMILES O=C1CC(CN1)N1CC1: local variable 'descriptors' referenced before assignment
Error calculating Mordred descriptors for SMILES CC1OC2COCC2O1: local variable 'descriptors' referenced before assignment
Error calculating Mordred descriptors for SMILES COCC(=O)C(=O)OC: local variable 'descriptors' referenced before assignment
Error calculating Mordred descriptors for SMILES C1C=CCC(=O)CC1=O: local variable 'descriptors' referenced before assignment
Error calculating Mordred descriptors for SMILES NC1=CC(=O)C(CO)C1: local variable 'descriptors' referenced before assignment


In [10]:
from mordred import descriptors

# Создание калькулятора с использованием всех дескрипторов из Mordred
calc = Calculator(descriptors)

# Генератор списка для вычисления дескрипторов для каждой молекулы
molecules = [Chem.MolFromSmiles(smiles) for smiles in df['smiles']]

# Объединение исходного DataFrame с дескрипторами
df_with_descriptors = calc.pandas(molecules)

100%|██████████| 100/100 [00:09<00:00, 10.58it/s]


In [11]:
num_rows, num_columns = df.shape
print(df.head())

       mol_id              smiles        A        B        C      mu  alpha  \
0   gdb_80215   OC1C2OC(=N)C12C#C  2.30911  1.61767  1.19613  2.3968  71.38   
1   gdb_38562  C1C2OC3C4C=CC13C24  3.62779  2.04768  1.72903  1.3033  75.21   
2  gdb_128434    COC(=N)N1C=NN=C1  4.59666  1.11509  0.90251  4.6009  68.56   
3   gdb_46047    O=C1CC(CN1)N1CC1  4.51768  0.96556  0.82880  4.0801  76.78   
4   gdb_92629       CC1OC2COCC2O1  2.61928  1.61948  1.55335  1.2958  70.36   

     homo    lumo     gap         r2      zpve          u0        u298  \
0 -0.2467 -0.0006  0.2461  1037.3102  0.098952 -436.608941 -436.600959   
1 -0.2190  0.0061  0.2251   817.0529  0.138958 -384.614538 -384.608953   
2 -0.2660 -0.0334  0.2325  1220.8171  0.113877 -450.138295 -450.130258   
3 -0.2380  0.0362  0.2742  1365.3572  0.161073 -419.221121 -419.212883   
4 -0.2408  0.0839  0.3247   980.1780  0.160888 -460.175337 -460.167671   

         h298        g298      cv mordred_descriptors  
0 -436.600015 -436.64134

In [12]:
print(num_rows, num_columns)

100 18


In [13]:
#соеденим с нашим датасетом по smiles
merged_df = pd.concat([df, df_with_descriptors], axis=1)

In [14]:
df = merged_df

In [15]:
num_rows, num_columns = merged_df.shape
print(num_rows, num_columns)

100 1844


In [16]:
print(merged_df.head())

       mol_id              smiles        A        B        C      mu  alpha  \
0   gdb_80215   OC1C2OC(=N)C12C#C  2.30911  1.61767  1.19613  2.3968  71.38   
1   gdb_38562  C1C2OC3C4C=CC13C24  3.62779  2.04768  1.72903  1.3033  75.21   
2  gdb_128434    COC(=N)N1C=NN=C1  4.59666  1.11509  0.90251  4.6009  68.56   
3   gdb_46047    O=C1CC(CN1)N1CC1  4.51768  0.96556  0.82880  4.0801  76.78   
4   gdb_92629       CC1OC2COCC2O1  2.61928  1.61948  1.55335  1.2958  70.36   

     homo    lumo     gap  ...      SRW10     TSRW10          MW       AMW  \
0 -0.2467 -0.0006  0.2461  ...  10.202147  63.238281  123.032028  8.788002   
1 -0.2190  0.0061  0.2251  ...  10.679251  62.435261  120.057515  7.062207   
2 -0.2660 -0.0334  0.2325  ...   8.520986  50.281704  126.054161  8.403611   
3 -0.2380  0.0362  0.2742  ...   8.993552  57.938858  126.079313  6.635753   
4 -0.2408  0.0839  0.3247  ...   8.969669  54.291600  130.062994  6.845421   

   WPath  WPol  Zagreb1 Zagreb2  mZagreb1  mZagreb2  
0 

In [17]:
# Вычисление дескрипторов с помощью PubChem
df['pubchem_descriptors'] = df['smiles'].apply(get_pubchem_descriptors)

Error fetching descriptors for SMILES OC1C2OC(=N)C12C#C: identifier/cid cannot be None
Error fetching descriptors for SMILES C1C2OC3C4C=CC13C24: identifier/cid cannot be None
Error fetching descriptors for SMILES COC(=N)N1C=NN=C1: identifier/cid cannot be None
Error fetching descriptors for SMILES O=C1CC(CN1)N1CC1: identifier/cid cannot be None
Error fetching descriptors for SMILES CC1OC2COCC2O1: 'PUGREST.BadRequest'
Error fetching descriptors for SMILES COCC(=O)C(=O)OC: 'PUGREST.BadRequest'
Error fetching descriptors for SMILES C1C=CCC(=O)CC1=O: 'PUGREST.BadRequest'
Error fetching descriptors for SMILES NC1=CC(=O)C(CO)C1: identifier/cid cannot be None
Error fetching descriptors for SMILES CC1N=COCCC1O: identifier/cid cannot be None
Error fetching descriptors for SMILES CCC1OC1(C)C1CC1: identifier/cid cannot be None
Error fetching descriptors for SMILES CN1C2C3CC1C(=O)C23: identifier/cid cannot be None
Error fetching descriptors for SMILES CC12CC3C(CC1O)C23: identifier/cid cannot be No

In [18]:
# Получение дескрипторов из PubChem для каждой молекулы
pubchem_descriptors = [get_pubchem_descriptors(smiles) for smiles in df['smiles']]

Error fetching descriptors for SMILES OC1C2OC(=N)C12C#C: identifier/cid cannot be None
Error fetching descriptors for SMILES C1C2OC3C4C=CC13C24: identifier/cid cannot be None
Error fetching descriptors for SMILES COC(=N)N1C=NN=C1: identifier/cid cannot be None
Error fetching descriptors for SMILES O=C1CC(CN1)N1CC1: identifier/cid cannot be None
Error fetching descriptors for SMILES CC1OC2COCC2O1: 'PUGREST.BadRequest'
Error fetching descriptors for SMILES COCC(=O)C(=O)OC: 'PUGREST.BadRequest'
Error fetching descriptors for SMILES C1C=CCC(=O)CC1=O: 'PUGREST.BadRequest'
Error fetching descriptors for SMILES NC1=CC(=O)C(CO)C1: identifier/cid cannot be None
Error fetching descriptors for SMILES CC1N=COCCC1O: identifier/cid cannot be None
Error fetching descriptors for SMILES CCC1OC1(C)C1CC1: identifier/cid cannot be None
Error fetching descriptors for SMILES CN1C2C3CC1C(=O)C23: identifier/cid cannot be None
Error fetching descriptors for SMILES CC12CC3C(CC1O)C23: identifier/cid cannot be No

In [19]:
# Объединение merged_df DataFrame с дескрипторами из PubChem
df_with_pubchem_descriptors = pd.concat([merged_df, pd.DataFrame(pubchem_descriptors)], axis=1)

In [20]:
# Вывод первых нескольких строк с дескрипторами из PubChem
print(df_with_pubchem_descriptors.head())

       mol_id              smiles        A        B        C      mu  alpha  \
0   gdb_80215   OC1C2OC(=N)C12C#C  2.30911  1.61767  1.19613  2.3968  71.38   
1   gdb_38562  C1C2OC3C4C=CC13C24  3.62779  2.04768  1.72903  1.3033  75.21   
2  gdb_128434    COC(=N)N1C=NN=C1  4.59666  1.11509  0.90251  4.6009  68.56   
3   gdb_46047    O=C1CC(CN1)N1CC1  4.51768  0.96556  0.82880  4.0801  76.78   
4   gdb_92629       CC1OC2COCC2O1  2.61928  1.61948  1.55335  1.2958  70.36   

     homo    lumo     gap  ...          MW       AMW  WPath  WPol  Zagreb1  \
0 -0.2467 -0.0006  0.2461  ...  123.032028  8.788002     80    10     54.0   
1 -0.2190  0.0061  0.2251  ...  120.057515  7.062207     66     6     68.0   
2 -0.2660 -0.0334  0.2325  ...  126.054161  8.403611     89     8     40.0   
3 -0.2380  0.0362  0.2742  ...  126.079313  6.635753     86     8     48.0   
4 -0.2408  0.0839  0.3247  ...  130.062994  6.845421     79     8     48.0   

   Zagreb2  mZagreb1  mZagreb2 pubchem_descriptors     0

# Применение методов отбора признаков

In [21]:
# Метод отбора на основе корреляции Пирсона
selector_pearson = SelectKBest(score_func=f_regression, k=500)

In [22]:
# Преобразование имен столбцов в строковый тип данных
df_with_pubchem_descriptors.columns = df_with_pubchem_descriptors.columns.astype(str)

In [23]:
print(df_with_pubchem_descriptors.dtypes.unique())

[dtype('O') dtype('float64') dtype('int64') dtype('bool')]


In [24]:
# Удаление столбцов с типом данных 'O' (строковых столбцов)
df_with_pubchem_descriptors_numeric = df_with_pubchem_descriptors.select_dtypes(exclude=['object'])

In [25]:
# Удаление столбца 'gap'
df_with_pubchem_descriptors_numeric.drop(columns=['gap'], inplace=True)


In [26]:
# Применение метода отбора признаков на основе корреляции Пирсона
selected_features_pearson = selector_pearson.fit_transform(df_with_pubchem_descriptors_numeric, df_with_pubchem_descriptors['gap'])
selected_feature_names_pearson = df_with_pubchem_descriptors_numeric.columns[selector_pearson.get_support()]

In [27]:
gap_counts = df['gap'].value_counts(normalize=False)
print(gap_counts)

gap
0.2097    3
0.2046    2
0.2161    2
0.2608    2
0.2155    2
         ..
0.1847    1
0.2638    1
0.3104    1
0.2522    1
0.2479    1
Name: count, Length: 94, dtype: int64


In [28]:
# Применение метода отбора признаков на основе взаимной информации
selector_mutual_info = SelectKBest(score_func=mutual_info_regression, k=500)

In [29]:
# Удаление столбцов с типом данных 'O' (строковых столбцов)
df_with_pubchem_descriptors_numeric = df_with_pubchem_descriptors.select_dtypes(exclude=['object'])


In [30]:
# Удаление столбцов 'gap' и столбца с идентификаторами молекул
df_with_pubchem_descriptors_numeric.drop(columns=['gap'], inplace=True)

In [31]:
# Применение метода отбора признаков на основе взаимной информации
selected_features_mutual_info = selector_mutual_info.fit_transform(df_with_pubchem_descriptors_numeric, df_with_pubchem_descriptors['gap'])
selected_feature_names_mutual_info = df_with_pubchem_descriptors_numeric.columns[selector_mutual_info.get_support()]

In [32]:
# Создание DataFrame с выбранными признаками и таргетом gap для корреляции Пирсона
selected_features_df_pearson = pd.DataFrame(selected_features_pearson, columns=selected_feature_names_pearson)
selected_features_df_pearson['gap'] = df_with_pubchem_descriptors['gap']

In [33]:
# Создание DataFrame с выбранными признаками и таргетом gap для взаимной информации
selected_features_df_mutual_info = pd.DataFrame(selected_features_mutual_info, columns=selected_feature_names_mutual_info)
selected_features_df_mutual_info['gap'] = df_with_pubchem_descriptors['gap']

In [35]:
# Сохранение датасетов с выбранными признаками
selected_features_df_pearson.to_csv('pearson_data.csv', index=False)
selected_features_df_mutual_info.to_csv('mutual_info)data.csv', index=False)

In [36]:
# Объединение выбранных признаков от метода отбора признаков на основе корреляции Пирсона и взаимной информации
selected_features_combined = pd.concat([selected_features_df_pearson, selected_features_df_mutual_info], axis=1)

In [37]:
# Сохранение объединенного датасета с выбранными признаками
selected_features_combined.to_csv('mesh_data.csv', index=False)